In [1]:
from langchain_ollama import OllamaLLM
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate

In [2]:
from langchain.schema import HumanMessage

In [3]:
llm = OllamaLLM(
    model="llama3", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), temperature = "0.1"
)

In [16]:
#Загрузка библиотек. Если не хватает библиотеки, то гуглить но почти всегда спасает pip install имя_пакета
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader

from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from chromadb.config import Settings
from langchain.vectorstores import Chroma

#from langchain.embeddings  import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# Загрузка файла pdf
loader = PyPDFLoader("/Users/alexeyvaganov/doc/Работа/Технология_корпоративного_внедрения/Организационная структура/Руководство по управлению проектами Часть 3 Организационная структура ТKV-0-001 v2.3.pdf")
documents = loader.load()

#loader = PyPDFDirectoryLoader("/Users/alexeyvaganov/doc/Работа/ИТ Практика/tz_arenda_pdf/")
#documents = loader.load()

# Определения принципа деления текста на части
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

# Делим текст на части
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")



Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 78 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)
Ignoring wrong pointing object 83 0 (offset 0)
Ignoring wrong 

Total documents: 74


In [17]:
print(type(documents))
print(documents[0].page_content)
print(documents[0].metadata)
print("-----")
print(documents[73].page_content)
print(documents[73].metadata)

<class 'list'>
Москва  2020  Подготовлено: Дата: Версия: Индекс документа: Морозова Е.А. 15.12.2021 2.3 TKV-0-001 Проверено: Дата: Заменяет: Страниц всего: Багмут Т.Ю. 17.12.2021 2.2 21 Утверждено: Дата: Действует для: Копия: Слесаренко М.И. 20.12.2021 1С:Франчайзи <Название фирмы> Оригинал       1С:Технология корпоративного внедрения  Руководство по управлению проектами. Часть 3. «Организационная структура»  Версия 2.3
{'source': '/Users/alexeyvaganov/doc/Работа/Технология_корпоративного_внедрения/Организационная структура/Руководство по управлению проектами Часть 3 Организационная структура ТKV-0-001 v2.3.pdf', 'page': 0}
-----
Страница    21 из 21  
В области функционального проектирования Обеспечивают полную и качественную реализацию методологических и регламентных документов функциональной области. Консультирует по сложным вопросам учета. Осуществляет взаимодействие с разработчиками программного продукта.  В области системной архитектуры Обеспечивает достижение оптимальной функцио

In [6]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
# Выбираем эмбединг (чтобы можно было перевести текст в вектор)
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

/Users/alexeyvaganov/miniforge3/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/alexeyvaganov/miniforge3/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
#  Загрузка в базу данных
db = Chroma.from_documents(
    documents,
    hf_embeddings_model,
    client_settings=Settings(anonymized_telemetry=False),
)

In [45]:
prompt = "Кто выволняет тестирование ИС ?"

In [46]:
data = db.similarity_search(prompt,k=4)
question = f"Используя эти данные: {data}. Ответь на русском языке этот запрос: {prompt}"
llm.invoke([HumanMessage(content=question)])

В соответствии с данными, ответом является: Тестировщик (11). Он выполняет внутреннее тестирование ИС в соответствии со сценариями тестирования, подготовленными аналитиками.

'В соответствии с данными, ответом является: Тестировщик (11). Он выполняет внутреннее тестирование ИС в соответствии со сценариями тестирования, подготовленными аналитиками.'

In [44]:
data

[Document(metadata={'page': 17, 'source': '/Users/alexeyvaganov/doc/Работа/Технология_корпоративного_внедрения/Организационная структура/Руководство по управлению проектами Часть 3 Организационная структура ТKV-0-001 v2.3.pdf'}, page_content='в ИС. Разрабатывает контрольные примеры и сценарии тестирования в части функциональных требований к ИС.  Разрабатывает проектную документацию по функционалу ИС, интеграции, миграции данных, распределению ролей и полномочий в ИС. 9.  Технический писатель Разрабатывает и актуализирует проектную документацию, входящую в состав результатов проекта. 10.  Программист Выполняет разработку/доработку программного продукта, взятого в качестве основы построения ИС, согласно постановке задачи аналитика. Взаимодействует с аналитиком (постановщиком) по вопросам к доработкам ИС. Разрабатывает проектную документацию по доработкам ИС: спецификации на разработку/доработку программного продукта. 11.  Тестировщик Выполняет внутреннее тестирование ИС в соответствии со

# УСТАНОВКА ПАКЕТОВ